<a href="https://colab.research.google.com/github/cazathoth/tfm/blob/main/tfm_jvasquez_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#descarga librerias.
install.packages("tidyverse")
install.packages("arrow")
install.packages("lubridate")
#install.packages('devtools')
#devtools::install_github('catboost/catboost', subdir = 'catboost/R-package')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘assertthat’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
#cargo librerias a utilizar.
library("tidyverse")
library("arrow")
library("lubridate")
#library("catboost")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp




In [3]:
#creo directorio y descargo dataset con datos de urgencias respiratorias
dir.create("descargas")
url <- "https://datos.gob.cl/dataset/606ef5bb-11d1-475b-b69f-b980da5757f4/resource/ae6c9887-106d-4e98-8875-40bf2b836041/download/at_urg_respiratorio_semanal.parquet"
nombre_temporal <- "descargas/at_urg_respiratorio_semanal.parquet"
download.file(url, nombre_temporal)

In [4]:
#Descargo datos meteorologicos- temperatura
url <- "https://raw.githubusercontent.com/cazathoth/data/9ed2ff302d64b81e79eab693e77d12f805ed65ba/MERGE_XXXX_DiarioTs_.csv"
nombre_temporal <- "descargas/MERGE_XXXX_DiarioTs_.csv"
download.file(url, nombre_temporal)

In [5]:
#Descargo datos meteorologicos- precipitaciones
url <- "https://raw.githubusercontent.com/cazathoth/data/43bad08be793c38c993d524d182dcb2597e14714/MERGE_DiarioRR.csv"
nombre_temporal <- "descargas/MERGE_DiarioRR.csv"
download.file(url, nombre_temporal)

In [6]:
#Descargo zip datos meteorologicos- humedad
url <- "https://github.com/cazathoth/data/raw/69c13d02f49cd04ee16ab8063f872e2aa9b2632a/MERGE_Humedad.zip"
nombre_temporal <- "descargas/MERGE_Humedad.zip"
download.file(url, nombre_temporal)
#Descomprimo archivo zip
unzip("/content/descargas/MERGE_Humedad.zip", exdir = "/content/descargas/")

In [422]:
#Descargo datos material particulado
url <- "https://raw.githubusercontent.com/cazathoth/data/6b00e8ef9b16a1802825e7914f998a6dd5e8f486/MERGE_MP.csv"
nombre_temporal <- "descargas/MERGE_MP.csv"
download.file(url, nombre_temporal)

In [741]:
# Creo data frame urgencias y filtro por comuna de Santiago, La Serena y Concepción, luego cargo los datos
df_urgr_esp_semanal <- read_parquet("descargas/at_urg_respiratorio_semanal.parquet", col_select = c ('EstablecimientoCodigo','ComunaCodigo', COMUNA='ComunaGlosa', 'Anio', NuSemana='SemanaEstadistica','OrdenCausa','Causa','NumTotal'))
df_urgr_esp_semanal <- df_urgr_esp_semanal %>% filter (ComunaCodigo %in% c(4101, 8101, 13101) & OrdenCausa ==3)
df_urgr_esp_semanal <- df_urgr_esp_semanal %>% rename(COMUNA = ComunaGlosa, NuSemana = SemanaEstadistica)
head(df_urgr_esp_semanal,3)

EstablecimientoCodigo,ComunaCodigo,COMUNA,Anio,NuSemana,OrdenCausa,Causa,NumTotal
<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<int>
110110,13101,Santiago,2023,50,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,0
111100,13101,Santiago,2021,52,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,0
110100,13101,Santiago,2017,46,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,68


In [742]:
#ano_base <- "2014-01-01"
#df_datos_temperatura <- read_delim("descargas/MERGE_XXXX_DiarioTs_.csv", delim=";",col_select = c('CodigoNacional','momento','MediaCli_Valor','Maxima_Valor','Minima_Valor'), col_types = cols(momento = col_date(format = "%d-%m-%Y %H:%M:%S")))
#df_datos_temperatura <- df_datos_temperatura %>% filter (momento > ano_base)
#head(df_datos_temperatura,15)

In [743]:
#Cargo data frame temperatura y le cargo datos
#df_datos_temperatura <- read.csv("descargas/MERGE_XXXX_DiarioTs_.csv", header= TRUE, sep=";")
df_datos_temperatura <- read_delim("descargas/MERGE_XXXX_DiarioTs_.csv", delim=";",col_select = c('CodigoNacional','momento','MediaCli_Valor','Maxima_Valor','Minima_Valor'))
head(df_datos_temperatura,3)

Rows: 74539 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (1): momento
dbl (4): CodigoNacional, MediaCli_Valor, Maxima_Valor, Minima_Valor

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


CodigoNacional,momento,MediaCli_Valor,Maxima_Valor,Minima_Valor
<dbl>,<chr>,<dbl>,<dbl>,<dbl>
290004,01-01-1954 12:00:00,16.5,20.6,12.3
290004,02-01-1954 12:00:00,17.1,20.4,13.9
290004,03-01-1954 12:00:00,15.1,20.0,10.2


In [744]:
#Cargo data frame humedad y le cargo datos
df_datos_Humedad <- read.csv("descargas/MERGE_Humedad.csv", header= TRUE, sep=";")
head(df_datos_Humedad,3)

,CodigoNacional,momento,HR_Valor
,<int>,<chr>,<dbl>
1,290004,01-01-1954 10:00:00,NA
2,290004,01-01-1954 11:00:00,NA
3,290004,01-01-1954 12:00:00,NA


In [745]:
#Cargo data frame precipitaciones y le cargo datos
#df_datos_Precip <- read.csv("descargas/MERGE_DiarioRR.csv", header= TRUE, sep=";")
df_datos_Precip <- read_delim("descargas/MERGE_DiarioRR.csv", delim=";",col_select = c('CodigoNacional','momento','Total_Valor'))
head(df_datos_Precip,3)

Rows: 66161 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (1): momento
dbl (2): CodigoNacional, Total_Valor

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


CodigoNacional,momento,Total_Valor
<dbl>,<chr>,<dbl>
290004,01-01-1954 12:00:00,0
290004,02-01-1954 12:00:00,0
290004,03-01-1954 12:00:00,0


In [746]:
#Cargo data frame material particulado y le cargo datos
#df_datos_mp25 <- read.csv("descargas/MERGE_MP.csv", header= TRUE, sep=";")
df_datos_mp25 <- read_delim("descargas/MERGE_MP.csv", delim=";",col_select = c('COMUNA', momento = 'FECHA(YYYYMMDD)', registrosValidados='Registros validados', registrosPreliminares='Registros preliminares', registrosNoValidados='Registros no validados' ))
 df_datos_mp25 <-df_datos_mp25%>%unite(totalMP25, c(registrosValidados,registrosPreliminares, registrosNoValidados), na.rm = TRUE)
 df_datos_mp25$totalMP25 <- as.double(df_datos_mp25$totalMP25)
head(df_datos_mp25,3)

Rows: 18476 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (1): COMUNA
dbl (4): Registros validados, Registros preliminares, Registros no validados...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


COMUNA,momento,totalMP25
<chr>,<dbl>,<dbl>
Concepcion,20090812,390.2810
Concepcion,20090813,68.5317
Concepcion,20090814,319.4660


In [747]:
#formateo fechas en material particulado 2.5.
df_datos_mp25$momento <- format(as.Date(as.character(df_datos_mp25$momento),format="%Y%m%d"), format="%d-%m-%Y")
#formateo fechas en temperaturas.
df_datos_temperatura$momento<- format(as.Date(df_datos_temperatura$momento, format = "%d-%m-%Y %H:%M:%S"), format = "%d-%m-%Y")
#formateo fechas en precipitaciones
df_datos_Precip$momento<- format(as.Date(df_datos_Precip$momento, format = "%d-%m-%Y %H:%M:%S"), format = "%d-%m-%Y")
#formateo fechas en humedad
df_datos_Humedad$momento<- format(as.Date(df_datos_Humedad$momento, format = "%d-%m-%Y %H:%M:%S"), format = "%d-%m-%Y")

In [748]:
#Limpio filas con NA en todos los dataframe y chequeo contando posibles NA
#df_urgr_esp_semanal <- na.omit(df_urgr_esp_semanal)
#df_datos_temperatura <- na.omit(df_datos_temperatura)
#df_datos_Precip <- na.omit(df_datos_Precip)
#df_datos_Humedad <- na.omit(df_datos_Humedad)
#df_datos_mp25 <- na.omit(df_datos_mp25)
sum(!complete.cases(df_urgr_esp_semanal))
sum(!complete.cases(df_datos_temperatura))
sum(!complete.cases(df_datos_Precip))
sum(!complete.cases(df_datos_Humedad))
sum(!complete.cases(df_datos_mp25))

[1] 0

[1] 10026

[1] 134

[1] 18269

[1] 1183

In [749]:
# agrupo urgencias respiratorias por año, semana y comuna y sumo el total de urgencias para la comuna.
df_urgr_esp_semanal <-df_urgr_esp_semanal %>% group_by (COMUNA, Anio, NuSemana) %>% summarise(NumTotalUrg = sum(NumTotal, na.rm = TRUE)) %>%filter(Anio > 2013 & Anio < 2024)
df_urgr_esp_semanal <- as.data.frame(df_urgr_esp_semanal)
df_urgr_esp_semanal$COMUNA[df_urgr_esp_semanal$COMUNA == "La Serena"] <- "LaSerena"
df_urgr_esp_semanal$COMUNA[df_urgr_esp_semanal$COMUNA == "Concepción"] <- "Concepcion"
head(df_urgr_esp_semanal,3)

`summarise()` has grouped output by 'COMUNA', 'Anio'. You can override using
the `.groups` argument.


,COMUNA,Anio,NuSemana,NumTotalUrg
,<chr>,<int>,<int>,<int>
1,Concepcion,2014,1,574
2,Concepcion,2014,2,965
3,Concepcion,2014,3,763


In [750]:
#obtengo n° de semana de las fechas de mediciones (momento) y agrego columna al data frame temperatura
semana<- epiweek(dmy(df_datos_temperatura$momento))
ano <- year(dmy(df_datos_temperatura$momento))
difTemp <- df_datos_temperatura$Maxima_Valor-df_datos_temperatura$Minima_Valor
df_datos_temperatura <- cbind(df_datos_temperatura, NuSemana = semana)
df_datos_temperatura <- cbind(df_datos_temperatura, Anio = ano)
df_datos_temperatura <- cbind(df_datos_temperatura, difTemp = difTemp)
head(df_datos_temperatura,3)

,CodigoNacional,momento,MediaCli_Valor,Maxima_Valor,Minima_Valor,NuSemana,Anio,difTemp
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,290004,01-01-1954,16.5,20.6,12.3,52,1954,8.3
2,290004,02-01-1954,17.1,20.4,13.9,52,1954,6.5
3,290004,03-01-1954,15.1,20.0,10.2,1,1954,9.8


In [751]:
# Obtengo la media y mediana, de la temperatura para cada semana en cada año.
df_datos_temperatura_semanal <- df_datos_temperatura %>% group_by (CodigoNacional,Anio, NuSemana) %>% summarise(medianaClim = median(MediaCli_Valor, na.rm = TRUE),  difMaxMin = median(difTemp, na.rm = TRUE))
#Convierto agrupación con media y mediana a data frame
df_datos_temperatura_semanal <- as.data.frame (df_datos_temperatura_semanal)
#filtro por todo aquellos años >= a 2014
df_datos_temperatura_semanal <- df_datos_temperatura_semanal %>% filter(Anio > 2013 & Anio < 2024)
head(df_datos_temperatura_semanal,3)



`summarise()` has grouped output by 'CodigoNacional', 'Anio'. You can override
using the `.groups` argument.


,CodigoNacional,Anio,NuSemana,medianaClim,difMaxMin
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,290004,2014,1,17.6,7.8
2,290004,2014,2,17.4,7.5
3,290004,2014,3,17.8,6.9


In [752]:
#obtengo n° de semana de las fechas de mediciones (momento) y agrego columna al data frame humedad
semana<-epiweek(dmy(df_datos_Humedad$momento))
ano <- year(dmy(df_datos_Humedad$momento))
df_datos_Humedad <- cbind(df_datos_Humedad, NuSemana = semana)
df_datos_Humedad <- cbind(df_datos_Humedad, Anio = ano)
head(df_datos_Humedad,3)

,CodigoNacional,momento,HR_Valor,NuSemana,Anio
,<int>,<chr>,<dbl>,<dbl>,<dbl>
1,290004,01-01-1954,NA,52,1954
2,290004,01-01-1954,NA,52,1954
3,290004,01-01-1954,NA,52,1954


In [753]:
# Obtengo la media y mediana, de la temperatura para cada semana en cada año.
df_datos_Humedad_semanal <- df_datos_Humedad %>% group_by (CodigoNacional,Anio, NuSemana) %>% summarise(Mediana_HR_Valor = median(HR_Valor, na.rm = TRUE))
#Convierto agrupación con media y mediana a data frame
df_datos_Humedad_semanal <- as.data.frame (df_datos_Humedad_semanal)
#filtro por todo aquellos años >= a 2014
df_datos_Humedad_semanal <- df_datos_Humedad_semanal %>% filter(Anio > 2013 & Anio < 2024)
head(df_datos_Humedad_semanal,3)

`summarise()` has grouped output by 'CodigoNacional', 'Anio'. You can override
using the `.groups` argument.


,CodigoNacional,Anio,NuSemana,Mediana_HR_Valor
,<int>,<dbl>,<dbl>,<dbl>
1,290004,2014,1,76.5
2,290004,2014,2,81.5
3,290004,2014,3,72.5


In [754]:
#obtengo n° de semana de las fechas de mediciones (momento) y agrego columna al data frame precipitaciones
semana<-epiweek(dmy(df_datos_Precip$momento))
ano <- year(dmy(df_datos_Precip$momento))
df_datos_Precip <- cbind(df_datos_Precip, NuSemana = semana)
df_datos_Precip <- cbind(df_datos_Precip, Anio = ano)
head(df_datos_Precip,3)

,CodigoNacional,momento,Total_Valor,NuSemana,Anio
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,290004,01-01-1954,0,52,1954
2,290004,02-01-1954,0,52,1954
3,290004,03-01-1954,0,1,1954


In [755]:
# Agrupo y obtengo la suma de precipitaciones para cada semana en cada año.
df_datos_Precip_semanal <- df_datos_Precip %>% group_by (CodigoNacional,Anio, NuSemana) %>% summarise(Total_Valor_semPP = sum(Total_Valor, na.rm = TRUE))
#Convierto agrupación con media y mediana a data frame
df_datos_Precip_semanal <- as.data.frame (df_datos_Precip_semanal)
#filtro por todo aquellos años >= a 2014
df_datos_Precip_semanal <- df_datos_Precip_semanal %>% filter(Anio > 2013 & Anio < 2024)
head(df_datos_Precip_semanal,3)

`summarise()` has grouped output by 'CodigoNacional', 'Anio'. You can override
using the `.groups` argument.


,CodigoNacional,Anio,NuSemana,Total_Valor_semPP
,<dbl>,<dbl>,<dbl>,<dbl>
1,290004,2014,1,0
2,290004,2014,2,0
3,290004,2014,3,0


In [756]:
#obtengo n° de semana de las fechas de mediciones (momento) y agrego columna al data frame mp25
semana<-epiweek(dmy(df_datos_mp25$momento))
ano <- year(dmy(df_datos_mp25$momento))
df_datos_mp25 <- cbind(df_datos_mp25, NuSemana = semana)
df_datos_mp25 <- cbind(df_datos_mp25, Anio = ano)
head(df_datos_mp25,3)

,COMUNA,momento,totalMP25,NuSemana,Anio
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,Concepcion,12-08-2009,390.2810,32,2009
2,Concepcion,13-08-2009,68.5317,32,2009
3,Concepcion,14-08-2009,319.4660,32,2009


In [757]:
# Agrupo y obtengo la suma de precipitaciones para cada semana en cada año.
df_datos_mp25_semanal <- df_datos_mp25 %>% group_by (COMUNA,Anio, NuSemana) %>% summarise(Total_Valor_semMP25 = sum(totalMP25, na.rm = TRUE))
#Convierto agrupación con media y mediana a data frame
df_datos_mp25_semanal <- as.data.frame (df_datos_mp25_semanal)
#filtro por todo aquellos años >= a 2014
df_datos_mp25_semanal <- df_datos_mp25_semanal %>% filter(Anio > 2013 & Anio < 2024)
head(df_datos_mp25_semanal,3)

`summarise()` has grouped output by 'COMUNA', 'Anio'. You can override using
the `.groups` argument.


,COMUNA,Anio,NuSemana,Total_Valor_semMP25
,<chr>,<dbl>,<dbl>,<dbl>
1,Concepcion,2014,1,48.20000
2,Concepcion,2014,2,73.69997
3,Concepcion,2014,3,72.09999


In [758]:
count(df_urgr_esp_semanal)
write.csv (df_urgr_esp_semanal, "prueba.csv")
count(df_datos_temperatura_semanal)
write.csv (df_datos_temperatura_semanal, "prueba2.csv")
count(df_datos_Precip_semanal)
count(df_datos_Humedad_semanal)
count(df_datos_mp25_semanal)

n
<int>
1572


n
<int>
1572


n
<int>
1572


n
<int>
1572


n
<int>
1572


In [759]:
df_variable <-merge(df_datos_temperatura_semanal, df_datos_Humedad_semanal, by = c("CodigoNacional","Anio","NuSemana"))
df_variable <- merge(df_variable, df_datos_Precip_semanal, by = c("CodigoNacional","Anio","NuSemana"))
head(df_variable,10)

,CodigoNacional,Anio,NuSemana,medianaClim,difMaxMin,Mediana_HR_Valor,Total_Valor_semPP
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,290004,2014,1,17.6,7.8,76.5,0.0
2,290004,2014,10,15.8,8.4,82.0,0.0
3,290004,2014,11,14.9,6.6,84.0,0.0
4,290004,2014,12,14.3,6.1,86.5,0.2
5,290004,2014,13,14.4,4.2,85.5,0.6
6,290004,2014,14,14.2,7.7,80.0,0.0
7,290004,2014,15,14.1,8.9,77.0,0.0
8,290004,2014,16,12.8,5.4,83.0,0.0
9,290004,2014,17,13.2,5.0,84.0,0.2


In [760]:
df_variable <- df_variable %>% mutate(COMUNA = ifelse(CodigoNacional == 290004, "LaSerena", ifelse (CodigoNacional == 360019, "Concepcion", ifelse (CodigoNacional == 330020, "Santiago", NA))))


In [761]:
df_variable <- merge(df_variable, df_urgr_esp_semanal, by = c("COMUNA","Anio","NuSemana"))
df_variable

COMUNA,Anio,NuSemana,CodigoNacional,medianaClim,difMaxMin,Mediana_HR_Valor,Total_Valor_semPP,NumTotalUrg
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
Concepcion,2014,1,360019,16.75,13.6,58.0,0.0,574
Concepcion,2014,10,360019,15.20,11.2,76.5,0.4,938
Concepcion,2014,11,360019,15.80,9.9,80.0,12.8,1270
Concepcion,2014,12,360019,15.30,12.0,72.0,0.8,1702
Concepcion,2014,13,360019,13.70,4.4,81.0,52.8,1450
Concepcion,2014,14,360019,12.80,9.2,84.0,0.0,1480
Concepcion,2014,15,360019,12.00,8.4,86.0,5.8,1317
Concepcion,2014,16,360019,12.70,11.2,73.0,0.0,1353
Concepcion,2014,17,360019,12.60,10.9,90.5,1.6,1400


In [762]:
df_variable <-df_variable %>% select(-CodigoNacional) %>% arrange(Anio,NuSemana)
head(df_variable,5)

,COMUNA,Anio,NuSemana,medianaClim,difMaxMin,Mediana_HR_Valor,Total_Valor_semPP,NumTotalUrg
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,Concepcion,2014,1,16.75,13.60,58.0,0,574
2,LaSerena,2014,1,17.60,7.80,76.5,0,396
3,Santiago,2014,1,21.95,19.15,32.0,0,295
4,Concepcion,2014,2,17.70,14.20,66.5,0,965
5,LaSerena,2014,2,17.40,7.50,81.5,0,694


In [763]:
df_datos_mp25_semanal %>% filter(Total_Valor_semMP25 == 0)

COMUNA,Anio,NuSemana,Total_Valor_semMP25
<chr>,<dbl>,<dbl>,<dbl>
Concepcion,2014,26,0
Concepcion,2014,27,0
Concepcion,2014,28,0
Concepcion,2014,29,0
Concepcion,2014,30,0
Concepcion,2014,31,0
Concepcion,2014,32,0
Concepcion,2014,33,0
Concepcion,2014,34,0


In [764]:
#Aqui estan todas las causas consideradas. La #3 es un total.
#df_urgr_esp_semanal %>% select(OrdenCausa, Causa)%>%distinct()%>%arrange(OrdenCausa)

In [765]:
#Filtro el dataset par las regiones 8,4 y 13, tambien la causa 3 que es solo totalizador de urgencias reportadas
#df_urgr_esp_semanal<- df_urgr_esp_semanal %>% filter (RegionCodigo %in% c(8,4,13) & OrdenCausa != 3)
#count(df_urgr_esp_semanal)
#head(df_urgr_esp_semanal,5)
#df_urgr_esp_semanal %>% filter (ComunaGlosa == 'La Serena' & Anio == 2023 & SemanaEstadistica == 47) %>%arrange (Anio)
#%>% select(ComunaCodigo, ComunaGlosa) %>% filter(ComunaGlosa == 'Santiago')

In [766]:
#Obtengo el total de urgencias reportadas en todos los años por las 3 regiones.
#df_urgr_esp_semanal%>% group_by(RegionGlosa) %>% summarise(Total = sum(NumTotal, na.rm = TRUE))

In [767]:
# Grafico la cantidad de casos por región
#df_plot1<-df_urgr_esp_semanal%>% group_by(RegionGlosa) %>% summarise(Total = sum(NumTotal, na.rm = TRUE))
#options(repr.plot.width = 10, repr.plot.height = 10)
#ggplot(df_plot1, aes(x=RegionGlosa, y= Total, fill= RegionGlosa)) + geom_bar(stat="identity") +
#scale_y_continuous(labels = scales::comma_format(accuracy = 1))+
#theme(axis.text.x = element_text(angle = 75, vjust = .5))+
#labs(x = "Región", y = "Total Urgencias", title = "Total Urgencias x Región")

In [768]:
# Obtengo cantidad de urgencias por año y region
#df_resumen <- df_urgr_esp_semanal %>%
#  group_by(Anio, RegionGlosa) %>%
#  summarise(Total = sum(NumTotal, na.rm = TRUE))
#  df_resumen

In [769]:
# Obtengo cantidad de urgencias por año, semana y region.
#df_resumen <- df_urgr_esp_semanal %>%
#  group_by(Anio, SemanaEstadistica, Causa) %>%
#  summarise(Total = sum(NumTotal, na.rm = TRUE))
#  df_resumen

In [770]:
#df_agrupado <- df_urgr_esp_semanal %>%
 # group_by(Anio) %>%
  #summarise(Total = sum(NumTotal, na.rm = TRUE))
#options(repr.plot.width = 15, repr.plot.height = 15)
#ggplot(df_agrupado, aes(x = factor(Anio), y = Total)) +
 # geom_bar(stat = "identity", fill = "steelblue") +
 # labs(x = "Año", y = "Total Urgencias", title = "Urgencias por Año") +
 # theme_minimal()